In [1]:
import csv

condition_mapping_file = "../../resources/CCSCM.csv"
procedure_mapping_file = "../../resources/CCSPROC.csv"
drug_file = "../../resources/ATC.csv"

condition_dict = {}
with open(condition_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        condition_dict[row['code']] = row['name'].lower()

procedure_dict = {}
with open(procedure_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        procedure_dict[row['code']] = row['name'].lower()

drug_dict = {}
with open(drug_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['level'] == '3':   #the prior cade was '5.0', changed to '3' to match graph_gen.ipynb
            drug_dict[row['code']] = row['name'].lower()


In [2]:
from tqdm import tqdm
import json

drug_ent = set()
drug_rel = set()

file_dir = "../../graphs/drug/ATC3"  #why in some files the level of the drug is 5 and in others 3? confusing.

for key in drug_dict.keys():
    file = f"{file_dir}/{key}.txt"
    with open(file=file, mode='r') as f:
        lines = f.readlines()
    
    for line in lines:
        parsed = line.split('\t')
        if len(parsed) == 3:
            h, r, t = line.split('\t')
            t = t[:-1]
            drug_ent.add(h)
            drug_ent.add(t)
            drug_rel.add(r)


drug_id2ent = {index: value for index, value in enumerate(drug_ent)}
drug_ent2id = {value: index for index, value in enumerate(drug_ent)}
drug_id2rel = {index: value for index, value in enumerate(drug_rel)}
drug_rel2id = {value: index for index, value in enumerate(drug_rel)}

out_file_id2ent = f"{file_dir}/id2ent.json"
out_file_ent2id = f"{file_dir}/ent2id.json"
out_file_id2rel = f"{file_dir}/id2rel.json"
out_file_rel2id = f"{file_dir}/rel2id.json"

with open(out_file_id2ent, 'w') as file:
    json.dump(drug_id2ent, file, indent=6)
with open(out_file_ent2id, 'w') as file:
    json.dump(drug_ent2id, file, indent=6)
with open(out_file_id2rel, 'w') as file:
    json.dump(drug_id2rel, file, indent=6)
with open(out_file_rel2id, 'w') as file:
    json.dump(drug_rel2id, file, indent=6)
    

In [3]:
import json

file_dir = "../../graphs/drug/ATC3"

file_id2ent = f"{file_dir}/id2ent.json"
file_ent2id = f"{file_dir}/ent2id.json"
file_id2rel = f"{file_dir}/id2rel.json"
file_rel2id = f"{file_dir}/rel2id.json"

with open(file_id2ent, 'r') as file:
    drug_id2ent = json.load(file)
with open(file_ent2id, 'r') as file:
    drug_ent2id = json.load(file)
with open(file_id2rel, 'r') as file:
    drug_id2rel = json.load(file)
with open(file_rel2id, 'r') as file:
    drug_rel2id = json.load(file)

In [4]:
import json
import pickle
import numpy as np
from tqdm import tqdm
from get_emb import embedding_retriever
import os
import glob

drug_ent_emb_dir = f"{file_dir}/drug_ent_emb"

if not os.path.exists(drug_ent_emb_dir):
    print(f"Creating directory {drug_ent_emb_dir}")
    os.makedirs(drug_ent_emb_dir)

for idx in tqdm(range(len(drug_id2ent))):
    ent = drug_id2ent[str(idx)]
    emb_filename = f"{drug_ent_emb_dir}/{ent}.json"
    if os.path.isfile(emb_filename):
        print(f"Skipping {ent}. Embedding file already exists.")
        continue
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    with open(emb_filename, "w") as f:
        json.dump(embedding.tolist(), f)
        print(f"Saved embedding for {ent} to {emb_filename}")

emb_files = glob.glob(f"{drug_ent_emb_dir}/*.json")
emb_list = []
for emb_file in emb_files:
    with open(emb_file, "r") as f:
        emb_list.append(np.array(json.load(f)))
stacked_embedding = np.vstack(emb_list)

emb_pkl = f"{file_dir}/entity_embedding.pkl"
with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

  0%|          | 0/20859 [00:00<?, ?it/s]

Skipping at dialysis center. Embedding file already exists.


  0%|          | 2/20859 [00:00<2:48:14,  2.07it/s]

Saved embedding for using hormonal contraceptives for systemic use to ../../graphs/drug/ATC3/drug_ent_emb/using hormonal contraceptives for systemic use.json


  0%|          | 3/20859 [00:01<2:41:31,  2.15it/s]

...

Saved embedding for x-ray contrast media to ../../graphs/drug/ATC3/drug_ent_emb/x-ray contrast media.json
Skipping individuals with known allergies. Embedding file already exists.


 11%|█         | 2281/20859 [09:04<1:05:48,  4.71it/s]

Saved embedding for provide fast-acting relief to ../../graphs/drug/ATC3/drug_ent_emb/provide fast-acting relief.json
Skipping flushing of the skin. Embedding file already exists.
Skipping used for chemoprophylaxis in individuals with sickle cell disease. Embedding file already exists.
Skipping temporary eye discomfort. Embedding file already exists.
Skipping kept away from eyes. Embedding file already exists.
Skipping pruritus severity. Embedding file already exists.


 11%|█         | 2286/20859 [09:05<1:13:50,  4.19it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../../graphs/drug/ATC3/drug_ent_emb/for the treatment of cutaneous adult T-cell leukemia/lymphoma.json'

In [5]:
# 加载所有entity的embedding，并将它们合并为一个numpy数组
emb_files = glob.glob(f"{drug_ent_emb_dir}/*.json")
emb_list = []
for emb_file in emb_files:
    with open(emb_file, "r") as f:
        emb_list.append(np.array(json.load(f)))
stacked_embedding = np.vstack(emb_list)

# 保存所有entity的embedding到pickle文件中
emb_pkl = f"{file_dir}/entity_embedding.pkl"
with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

In [6]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for drug entities
drug_ent_emb = []

for idx in tqdm(range(len(drug_id2ent))):
    ent = drug_id2ent[str(idx)]
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    drug_ent_emb.append(embedding)

stacked_embedding = np.vstack(drug_ent_emb)

emb_pkl = f"{file_dir}/entity_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

 71%|███████   | 14785/20859 [2:12:26<54:24,  1.86it/s]  


SSLError: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/embeddings (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1129)')))

In [16]:
import json
import pickle
import numpy as np
from tqdm import tqdm
from get_emb import embedding_retriever
import os
import glob

drug_ent_emb_dir = f"{file_dir}/drug_ent_emb"

if not os.path.exists(drug_ent_emb_dir):
    print(f"Creating directory {drug_ent_emb_dir}")
    os.makedirs(drug_ent_emb_dir)

for idx in tqdm(range(len(drug_id2ent))):
    ent = drug_id2ent[str(idx)]
    emb_filename = f"{drug_ent_emb_dir}/{ent}.json"
    if os.path.isfile(emb_filename):
        print(f"Skipping {ent}. Embedding file already exists.")
        continue
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    with open(emb_filename, "w") as f:
        json.dump(embedding.tolist(), f)
        print(f"Saved embedding for {ent} to {emb_filename}")

emb_files = glob.glob(f"{drug_ent_emb_dir}/*.json")
emb_list = []
for emb_file in emb_files:
    with open(emb_file, "r") as f:
        emb_list.append(np.array(json.load(f)))
stacked_embedding = np.vstack(emb_list)

emb_pkl = f"{file_dir}/entity_embedding.pkl"
with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

  2%|▏         | 507/20859 [00:00<00:04, 5068.74it/s]

Skipping combined with other procedures. Embedding file already exists.
Skipping used as a supplement. Embedding file already exists.
Skipping can be measured. Embedding file already exists.
Skipping drug dependence. Embedding file already exists.
Skipping the appearance of scars. Embedding file already exists.
Skipping Alzheimer's disease pathology. Embedding file already exists.
Skipping urinary dribbling. Embedding file already exists.
Skipping hormone therapy to express their gender identity. Embedding file already exists.
Skipping be applied as directed. Embedding file already exists.
Skipping treatment of ulcerative colitis. Embedding file already exists.
Skipping loss of sensation. Embedding file already exists.
Skipping adjustments in patients with psychiatric disorders. Embedding file already exists.
Skipping increased risk of infection. Embedding file already exists.
Skipping diuretic medications. Embedding file already exists.
Skipping kill cancer cells. Embedding file alrea

  3%|▎         | 568/20859 [00:01<01:11, 284.35it/s] 


FileNotFoundError: [Errno 2] No such file or directory: '../../graphs/drug/ATC3/drug_ent_emb/intestinal antiinflammatory/antiinfective agents.json'

In [9]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for drug relations
drug_rel_emb = []

for idx in tqdm(range(len(drug_id2rel))):
    rel = drug_id2rel[str(idx)]
    embedding = embedding_retriever(term=rel)
    embedding = np.array(embedding)
    drug_rel_emb.append(embedding)

stacked_embedding = np.vstack(drug_rel_emb)

emb_pkl = f"{file_dir}/relation_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

100%|██████████| 2283/2283 [22:39<00:00,  1.68it/s] 


--------------------

In [ ]:
import csv

condition_mapping_file = "../../resources/CCSCM.csv"
procedure_mapping_file = "../../resources/CCSPROC.csv"
drug_file = "../../resources/ATC.csv"

condition_dict = {}
with open(condition_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        condition_dict[row['code']] = row['name'].lower()

procedure_dict = {}
with open(procedure_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        procedure_dict[row['code']] = row['name'].lower()

drug_dict = {}
with open(drug_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['level'] == '3':   #the prior cade was '5.0', changed to '3' to match graph_gen.ipynb
            drug_dict[row['code']] = row['name'].lower()


In [ ]:
from tqdm import tqdm
import json

proc_ent = set()
proc_rel = set()

file_dir = "../../graphs/procedure/CCSPROC"

for key in procedure_dict.keys():
    file = f"{file_dir}/{key}.txt"
    with open(file=file, mode='r') as f:
        lines = f.readlines()
    
    for line in lines:
        parsed = line.split('\t')
        if len(parsed) == 3:
            h, r, t = line.split('\t')
            t = t[:-1]
            proc_ent.add(h)
            proc_ent.add(t)
            proc_rel.add(r)


proc_id2ent = {index: value for index, value in enumerate(proc_ent)}
proc_ent2id = {value: index for index, value in enumerate(proc_ent)}
proc_id2rel = {index: value for index, value in enumerate(proc_rel)}
proc_rel2id = {value: index for index, value in enumerate(proc_rel)}

out_file_id2ent = f"{file_dir}/id2ent.json"
out_file_ent2id = f"{file_dir}/ent2id.json"
out_file_id2rel = f"{file_dir}/id2rel.json"
out_file_rel2id = f"{file_dir}/rel2id.json"

with open(out_file_id2ent, 'w') as file:
    json.dump(proc_id2ent, file, indent=6)
with open(out_file_ent2id, 'w') as file:
    json.dump(proc_ent2id, file, indent=6)
with open(out_file_id2rel, 'w') as file:
    json.dump(proc_id2rel, file, indent=6)
with open(out_file_rel2id, 'w') as file:
    json.dump(proc_rel2id, file, indent=6)
    

In [ ]:
import json

file_dir = "../../graphs/procedure/CCSPROC"

file_id2ent = f"{file_dir}/id2ent.json"
file_ent2id = f"{file_dir}/ent2id.json"
file_id2rel = f"{file_dir}/id2rel.json"
file_rel2id = f"{file_dir}/rel2id.json"

with open(file_id2ent, 'r') as file:
    proc_id2ent = json.load(file)
with open(file_ent2id, 'r') as file:
    proc_ent2id = json.load(file)
with open(file_id2rel, 'r') as file:
    proc_id2rel = json.load(file)
with open(file_rel2id, 'r') as file:
    proc_rel2id = json.load(file)

In [ ]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for procedure entities
proc_ent_emb = []

for idx in tqdm(range(len(proc_id2ent))):
    ent = proc_id2ent[str(idx)]
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    proc_ent_emb.append(embedding)

stacked_embedding = np.vstack(proc_ent_emb)

emb_pkl = f"{file_dir}/entity_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

In [ ]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for procedure relations
proc_rel_emb = []

for idx in tqdm(range(len(proc_id2rel))):
    rel = proc_id2rel[str(idx)]
    embedding = embedding_retriever(term=rel)
    embedding = np.array(embedding)
    proc_rel_emb.append(embedding)

stacked_embedding = np.vstack(proc_rel_emb)

emb_pkl = f"{file_dir}/relation_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)